# Data Exploration - Individual Mobility

Import libraries and reading data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from itertools import product

from IPython.display import display
import time

Further cleaning the data

In [2]:
# new_nyc_df = pd.read_csv('cleaned_NYC_records_duration.csv')
# new_nyc_df['cleaned_UTC_time'] = pd.to_datetime(new_nyc_df["UTC_time"])
# new_nyc_df.loc[pd.isna(new_nyc_df.general_venue_category_name), 'general_venue_category_name'] = 'Others'

# new_nyc_write_df = new_nyc_df[['user_id', 'venue_id', 'venue_category_id', 'venue_category_name', 'general_venue_category_name', 'latitude',
#                                'longitude', 'UTC_time', 'duration']]
# new_nyc_write_df.to_csv('cleaned_NYC_records_duration.csv', index=False)

In [2]:
new_nyc_df = pd.read_csv('cleaned_NYC_records_duration.csv')
new_nyc_df['cleaned_UTC_time'] = pd.to_datetime(new_nyc_df["UTC_time"])
less_equal_2hours_df = new_nyc_df[(new_nyc_df.duration <= 2) & (new_nyc_df.duration != -1)]

months = [4, 5, 6]
less_than_2hrs_3months_df = less_equal_2hours_df.copy()
less_than_2hrs_3months_df['month'] = less_than_2hrs_3months_df['cleaned_UTC_time'].dt.month
less_than_2hrs_3months_df = less_than_2hrs_3months_df[less_than_2hrs_3months_df['month'].isin(months)]

In [3]:
def finding_days_users_less_than_2hrs_records_groupto(df):
    all_users_days_df = df.copy()
    all_users_days_df['date'] = all_users_days_df.apply(lambda x : x.cleaned_UTC_time.date(), axis=1)
    all_users_unique_days_df = all_users_days_df.groupby('user_id')['date'].nunique().reset_index(name='counts')
    all_users_unique_days_df = all_users_unique_days_df.sort_values(by=['counts'])
    return all_users_unique_days_df

In [4]:
user_days_df = finding_days_users_less_than_2hrs_records_groupto(less_than_2hrs_3months_df)
frequent_users_df = user_days_df[user_days_df['counts'] >= 50]

In [5]:
user_set_df = less_than_2hrs_3months_df[less_than_2hrs_3months_df.user_id.isin(frequent_users_df.user_id)]
user_set_df.groupby('user_id')['general_venue_category_name'].apply(lambda x: list(pd.unique(x))).reset_index(name='category_set')

,user_id,category_set
0,84,"[Eateries, Outdoor, Arts, Others, Shops, Enter..."
1,120,"[Eateries, Shops, Others, Entertainment, Relig..."
2,178,"[Eateries, Shops, Others, Fitness, Services, O..."
3,184,"[Eateries, Others, Arts, Services, Shops, Ente..."
4,185,"[Eateries, Arts, Others, Services, Outdoor, En..."
5,187,"[Others, Shops, Eateries, Arts, Outdoor, Servi..."
6,207,"[Others, Shops, Education, Services, Outdoor]"
7,280,"[Eateries, Shops, Services, Others, Education,..."
8,281,"[Shops, Eateries, Outdoor, Religious Places, O..."
9,293,"[Outdoor, Eateries, Others, Shops, Services, E..."


In [6]:
user_set_df = less_than_2hrs_3months_df[less_than_2hrs_3months_df.user_id.isin(frequent_users_df.user_id)]
a_df = user_set_df.groupby('user_id')['general_venue_category_name'].apply(lambda x: len(list(pd.unique(x)))).reset_index(name='category_set')
a_df.describe()

,user_id,category_set
count,34.000000,34.000000
mean,494.352941,7.323529
std,273.990697,2.211822
min,84.000000,2.000000
25%,284.000000,6.000000
50%,441.500000,7.000000
75%,701.750000,8.750000
max,990.000000,11.000000


In [7]:
user_set_count_df = less_than_2hrs_3months_df.groupby(['user_id', 'general_venue_category_name']).size().reset_index(name='counts')
user_set_count_df = user_set_count_df[user_set_count_df.user_id.isin(frequent_users_df.user_id)]
user_set_count_df

,user_id,general_venue_category_name,counts
355,84,Arts,80
356,84,Eateries,219
357,84,Education,10
358,84,Entertainment,10
359,84,Exhibitions,2
...,...,...,...
4527,974,Outdoor,85
4528,974,Shops,15
4602,990,Eateries,1
4603,990,Others,42


In [8]:
# Compute the neighbor index of the current index
def neighbors(index):
    N = len(index)
    for relative_index in product((0, -1), repeat=N): # generate combination of 0 and -1 with the same length as the index
        if not all(i == 0 for i in relative_index): # there is at least one entry that is subtracted by -1
            yield tuple(i + i_rel for i, i_rel in zip(index, relative_index))
            
def longestCommonSubSequenceOfN(sqs):
    numberOfSequences = len(sqs); # calculate the number of sequences input
    lengths = np.array([len(sequence) for sequence in sqs]); # compute the length of each sequence and save to an array
    incrLengths = lengths + 1; # add 1 to each length of sequence
    lengths = tuple(lengths); # change the lengths type from array to tuple
    inverseDistances = np.zeros(incrLengths); # make zero matrix as incrLengths size
    ranges = [tuple(range(1, length+1)) for length in lengths[::-1]]; # start from here, the author use 1 to represent the 0th element,
                                                                      # and the length is arranged in inverse order
        
    # For 3 sequences with lengths 3, 2, 1
    # The ranges becomes [(1), (1, 2), (1, 2, 3)]
    for tupleIndex in product(*ranges): 
        #print(tupleIndex)
        tupleIndex = tupleIndex[::-1]; # inverse the tupleIndex variable
        #print(tupleIndex)
        neighborIndexes = list(neighbors(tupleIndex)); # compute all possible neighbors index
        #print(neighborIndexes)
        operationsWithMisMatch = np.array([]);
        for neighborIndex in neighborIndexes:
            operationsWithMisMatch = np.append(operationsWithMisMatch, inverseDistances[neighborIndex]);
        operationsWithMatch = np.copy(operationsWithMisMatch);
        #print(operationsWithMatch)
        operationsWithMatch[-1] = operationsWithMatch[-1] + 1;
        chars = [sqs[i][neighborIndexes[-1][i]] for i in range(numberOfSequences)];
        #print(chars)
        #print(inverseDistances)
        if(all(elem == chars[0] for elem in chars)):
            inverseDistances[tupleIndex] = max(operationsWithMatch);
        else:
            inverseDistances[tupleIndex] = max(operationsWithMisMatch);
        # pdb.set_trace();

    print(inverseDistances)
#     subString = "";
    returnSequences = [];
    mainTupleIndex = lengths;
    while(all(ind > 0 for ind in mainTupleIndex)):
        neighborsIndexes = list(neighbors(mainTupleIndex));
        print(neighborsIndexes)
        anyOperation = False;
        for tupleIndex in neighborsIndexes:
            current = inverseDistances[mainTupleIndex];
            if(current == inverseDistances[tupleIndex]):
                mainTupleIndex = tupleIndex;
                anyOperation = True;
                break;
        if(not anyOperation):
#             subString += str(sqs[0][mainTupleIndex[0] - 1]);
            returnSequences.append(sqs[0][mainTupleIndex[0] - 1]);
            mainTupleIndex = neighborsIndexes[-1];
#     return subString[::-1];
    return returnSequences[::-1];

In [10]:
seq = [['A', 'B', 'C'], ['A', 'C'], ['A','C']]
longestCommonSubSequenceOfN(seq)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 1. 1.]
  [0. 1. 1.]]

 [[0. 0. 0.]
  [0. 1. 1.]
  [0. 1. 1.]]

 [[0. 0. 0.]
  [0. 1. 1.]
  [0. 1. 2.]]]
[(3, 2, 1), (3, 1, 2), (3, 1, 1), (2, 2, 2), (2, 2, 1), (2, 1, 2), (2, 1, 1)]
[(2, 1, 0), (2, 0, 1), (2, 0, 0), (1, 1, 1), (1, 1, 0), (1, 0, 1), (1, 0, 0)]
[(1, 1, 0), (1, 0, 1), (1, 0, 0), (0, 1, 1), (0, 1, 0), (0, 0, 1), (0, 0, 0)]


['A', 'C']

In [10]:
def generateLCSForUser(df, user_id):
    start_time = time.time()
    result_df = pd.DataFrame(columns=['user_id', 'LCS'])
    LCS_list = []
    user_df = df[df.user_id == user_id]
    sequences = np.array(user_df.category_set)
    LCS_list.append(longestCommonSubSequenceOfN(sequences[:32]))
    LCS_list.append(longestCommonSubSequenceOfN(sequences[32:]))
    print(longestCommonSubSequenceOfN(LCS_list))
    print("The process takes {} seconds".format(time.time() - start_time))

In [9]:
subsequence_all_df = user_set_df.copy()
subsequence_all_df['date'] = subsequence_all_df.apply(lambda x : x.cleaned_UTC_time.date(), axis=1)
subsequence_all_df = subsequence_all_df.groupby(['user_id', 'date'])['general_venue_category_name'].apply(lambda x: list(x)).reset_index(name='category_set')
subsequence_all_df['counts'] = subsequence_all_df['category_set'].apply(lambda x: len(x))

User id 187 has 50 records, just passing the threshold

In [12]:
# def generateLCSForUser187(df):
#     start_time = time.time()
#     result_df = pd.DataFrame(columns=['user_id', 'LCS'])
#     LCS_list = []
#     user_df = df[df.user_id == 187]
#     sequences = np.array(user_df.category_set)
#     LCS_list.append(longestCommonSubSequenceOfN(sequences[:32]))
#     LCS_list.append(longestCommonSubSequenceOfN(sequences[32:]))
#     print(longestCommonSubSequenceOfN(LCS_list))
#     print("The process takes {} seconds".format(time.time() - start_time))
#     new_row = {'user_id': user, 'LCS': longestCommonSubSequenceOfN(sequences)}
#     result_df.append(new_row, ignore_index=True)

In [13]:
sequences_187_df = subsequence_all_df[(subsequence_all_df.user_id == 187) & (subsequence_all_df.counts > 1)]
sequences_187_df.groupby(['counts']).size().reset_index(name='size')

,counts,size
0,2,3
1,3,14
2,4,13
3,5,8
4,6,3
5,7,1
6,8,2


In [14]:
sequences_187_df

,user_id,date,category_set,counts
314,187,2012-04-06,"[Others, Others, Others]",3
315,187,2012-04-07,"[Shops, Shops]",2
316,187,2012-04-09,"[Others, Others, Eateries, Others]",4
318,187,2012-04-12,"[Others, Others, Eateries, Others, Eateries]",5
319,187,2012-04-13,"[Others, Others, Eateries]",3
320,187,2012-04-15,"[Others, Shops]",2
321,187,2012-04-17,"[Others, Others, Eateries, Others, Others]",5
322,187,2012-04-18,"[Others, Others, Eateries, Others, Others]",5
323,187,2012-04-19,"[Others, Others, Eateries, Others]",4
324,187,2012-04-20,"[Others, Others, Eateries, Others]",4


In [15]:
exploring_df = sequences_187_df[(sequences_187_df.counts > 2) & (sequences_187_df.counts < 6)]
start_time = time.time()
LCS_lists = []
sequences = np.array(exploring_df.category_set)
for i in range(0, len(exploring_df), 4):
    if i+4 > len(sequences):
        LCS_lists.append(longestCommonSubSequenceOfN(sequences[i:]))
    else:
        LCS_lists.append(longestCommonSubSequenceOfN(sequences[i : i+4]))
print(LCS_lists)
LCS_lists = [x for x in LCS_lists if x != []]
print(longestCommonSubSequenceOfN(LCS_lists))
print("The process takes {} seconds".format(time.time() - start_time))

[['Others', 'Others'], ['Others', 'Others', 'Eateries', 'Others'], [], ['Others', 'Eateries', 'Others'], ['Others', 'Others'], ['Others', 'Others'], ['Others'], ['Others', 'Others'], ['Others', 'Others']]
['Others']
The process takes 2.6998815536499023 seconds


In [16]:
start_time = time.time()
LCS_lists = []
user_df = subsequence_all_df[subsequence_all_df.user_id == 187]
sequences = np.array(user_df.category_set)
for i in range(0, len(user_df), 4):
    if i+4 > len(sequences):
        LCS_lists.append(longestCommonSubSequenceOfN(sequences[i:]))
    else:
        LCS_lists.append(longestCommonSubSequenceOfN(sequences[i : i+4]))
print(LCS_lists)
LCS_lists = [x for x in LCS_lists if x != []]
print(longestCommonSubSequenceOfN(LCS_lists))
print("The process takes {} seconds".format(time.time() - start_time))

[[], ['Others'], ['Others'], ['Others', 'Others', 'Others'], [], ['Others', 'Others'], [], ['Others', 'Others', 'Eateries', 'Others'], ['Others', 'Others'], ['Others'], ['Others', 'Others', 'Eateries'], [], ['Others', 'Others', 'Others']]
['Others']
The process takes 6.190539121627808 seconds


Computing the most dominant sequences for all users

In [10]:
def compute_the_top_k_groups(df):
    temp_df = df.groupby(['counts']).size().reset_index(name='size')
    temp_df = temp_df.sort_values(by=['size'], ascending=False)
    print(temp_df)
    return np.array(temp_df.counts)

def compute_dominant_sequence(user_records_df):
    start_time = time.time()
    LCS_lists = []
    sequences = np.array(user_records_df.category_set)
    for i in range(0, len(user_records_df), 4):
        if i+4 > len(sequences):
            LCS_lists.append(longestCommonSubSequenceOfN(sequences[i:]))
        else:
            LCS_lists.append(longestCommonSubSequenceOfN(sequences[i : i+4]))
    LCS_lists = [x for x in LCS_lists if x != []]
    print(LCS_lists)
    LCS = longestCommonSubSequenceOfN(LCS_lists)
    elapsed_time = time.time() - start_time
    print(elapsed_time)
    return (LCS, elapsed_time)

def compute_dominant_sequences_for_users(df, frequent_users_df):
    result_df = pd.DataFrame(columns=['user_id', 'dominant_sequence', 'time (s)'])
    for user_id in frequent_users_df.user_id:
        print(user_id)
        user_records_df = df[(df.user_id == user_id) & (df.counts > 1)]
        topk_array = compute_the_top_k_groups(user_records_df)
        print(topk_array)
        topk_array = topk_array[:3]
        user_records_df = user_records_df[user_records_df.counts.isin(topk_array)]
        LCS, elapsed_time = compute_dominant_sequence(user_records_df)
        new_row = {'user_id': user_id, 'dominant_sequence': LCS, 'time (s)': elapsed_time}
        result_df = result_df.append(new_row, ignore_index=True)
        print()
    return result_df

def compute_LCS_for_users(df, frequent_users_df):
    result_df = pd.DataFrame(columns=['user_id', 'dominant_sequence', 'time (s)'])
    for user_id in frequent_users_df.user_id:
        print(user_id)
        user_records_df = df[(df.user_id == user_id)]
        LCS, elapsed_time = compute_dominant_sequence(user_records_df)
        new_row = {'user_id': user_id, 'dominant_sequence': LCS, 'time (s)': elapsed_time}
        result_df = result_df.append(new_row, ignore_index=True)
        print()
    return result_df

In [11]:
result_df = compute_dominant_sequences_for_users(subsequence_all_df, frequent_users_df)

187
   counts  size
1       3    14
2       4    13
3       5     8
0       2     3
4       6     3
6       8     2
5       7     1
[3 4 5 2 6 8 7]
[['Others', 'Others'], ['Others', 'Others', 'Eateries', 'Others'], ['Others', 'Eateries', 'Others'], ['Others', 'Others'], ['Others', 'Others'], ['Others'], ['Others', 'Others'], ['Others', 'Others']]
1.314002275466919

384
    counts  size
10      12     9
4        6     6
13      15     5
11      13     4
2        4     3
1        3     3
0        2     2
15      20     2
5        7     2
6        8     2
7        9     2
8       10     2
12      14     2
16      21     1
9       11     1
14      17     1
3        5     1
17      23     1
[12  6 15 13  4  3  2 20  7  8  9 10 14 21 11 17  5 23]
[['Eateries', 'Others', 'Eateries', 'Others', 'Eateries'], ['Others', 'Others', 'Others'], ['Others', 'Eateries'], ['Eateries', 'Services', 'Eateries'], ['Eateries', 'Eateries', 'Eateries', 'Services']]
11.438998699188232

448
    counts  size
0    

[['Eateries'], ['Eateries'], ['Eateries'], ['Entertainment', 'Eateries'], ['Eateries', 'Eateries'], ['Eateries'], ['Eateries', 'Eateries'], ['Eateries', 'Eateries']]
0.34696006774902344

443
   counts  size
1       3    13
0       2    10
2       4    10
4       6     6
5       7     5
3       5     4
6       8     3
7       9     2
8      11     2
[ 3  2  4  6  7  5  8  9 11]
[['Others'], ['Others', 'Others'], ['Eateries'], ['Eateries', 'Others'], ['Others'], ['Eateries'], ['Arts', 'Outdoor']]
0.1119985580444336

371
    counts  size
3        5     9
0        2     8
7        9     8
6        8     7
4        6     6
8       10     6
5        7     5
2        4     4
1        3     3
10      12     3
9       11     2
11      13     2
[ 5  2  9  8  6 10  7  4  3 12 11 13]
[['Others', 'Others'], ['Others'], ['Others'], ['Others'], ['Others'], ['Others', 'Others', 'Others', 'Eateries', 'Others']]
0.6509995460510254

354
    counts  size
0        2    10
3        5     8
1        3     7


In [12]:
result_df

,user_id,dominant_sequence,time (s)
0,187,[Others],1.314002
1,384,[],11.438999
2,448,[],0.199000
3,819,[Others],0.415029
4,280,[Eateries],0.054999
5,281,[],0.106999
6,742,[Others],0.373035
7,207,[],0.047000
8,184,[],0.070990
9,962,[Eateries],0.017995


In [85]:
# full_result_df = compute_LCS_for_users(subsequence_all_df, frequent_users_df)

187
2.4469873905181885

384
7874.523446083069

448
0.839989185333252

819
1.2560040950775146

280
1.17799711227417

281
0.3789703845977783

742
1.1339938640594482

207
0.05400514602661133

184
0.6480062007904053

962
0.011994361877441406

593
0.09199166297912598

652
0.049001216888427734

178
7.228996992111206

689
0.03799891471862793

706
5658.0927748680115

951
0.06699728965759277

527
5.999067544937134

943
0.16699981689453125

990
448.3742334842682

120
0.19499993324279785

974
1.9419965744018555

439
0.38100218772888184

396
0.17297053337097168

768
3.8649983406066895

440
0.7289927005767822

445
1.1159932613372803

443
0.7190287113189697

371
896.0312037467957

354
10.521871566772461

315


KeyboardInterrupt: 

In [108]:
user_ids = [187, 819, 280, 742, 962, 178, 689, 706, 990, 396, 445, 371, 354, 185]
full_result_df = pd.DataFrame(columns=['user_id', 'dominant_sequence', 'time (s)'])

for user_id in user_ids:
    print(user_id)
    user_records_df = subsequence_all_df[(subsequence_all_df.user_id == user_id)]
    LCS, elapsed_time = compute_dominant_sequence(user_records_df)
    new_row = {'user_id': user_id, 'dominant_sequence': LCS, 'time (s)': elapsed_time}
    full_result_df = full_result_df.append(new_row, ignore_index=True)
    print()

187
[['Others'], ['Others'], ['Others', 'Others', 'Others'], ['Others', 'Others'], ['Others', 'Others', 'Eateries', 'Others'], ['Others', 'Others'], ['Others'], ['Others', 'Others', 'Eateries'], ['Others', 'Others', 'Others']]
2.5120341777801514

819
[['Outdoor', 'Others'], ['Others', 'Others', 'Others'], ['Others'], ['Others'], ['Others', 'Others', 'Others'], ['Others'], ['Others'], ['Others', 'Others'], ['Others', 'Others']]
0.8380346298217773

280
[['Eateries', 'Eateries', 'Eateries'], ['Eateries'], ['Eateries'], ['Eateries', 'Eateries', 'Eateries', 'Eateries'], ['Eateries'], ['Eateries', 'Eateries'], ['Eateries', 'Eateries', 'Eateries', 'Eateries']]
0.8800337314605713

742
[['Others'], ['Others'], ['Others'], ['Others'], ['Others'], ['Others'], ['Others', 'Others'], ['Others'], ['Others'], ['Others'], ['Others'], ['Others']]
0.8250410556793213

962
[['Others'], ['Others'], ['Others']]
0.008000850677490234

178
[['Shops'], ['Shops'], ['Others'], ['Shops', 'Eateries', 'Eateries'], ['

In [109]:
full_result_df

,user_id,dominant_sequence,time (s)
0,187,[Others],2.512034
1,819,[Others],0.838035
2,280,[Eateries],0.880034
3,742,[Others],0.825041
4,962,[Others],0.008001
5,178,[],5.145028
6,689,[],0.021033
7,706,[],5013.850592
8,990,[Services],374.300593
9,396,[Outdoor],0.175022


In [48]:
user_id = 448
user_record_df = subsequence_all_df[subsequence_all_df.user_id == user_id]
user_record_df['categories'] = user_record_df['category_set'].apply(lambda x: list(pd.unique(x)))

print(user_record_df.categories)

categories = {}
for record in user_record_df.categories:
    for x in record:
        if x not in categories.keys():
            categories[x] = 1
        else:
            categories[x] = categories[x] + 1
            
print(categories)

1162            [Eateries, Shops, Others, Education]
1163                                      [Eateries]
1164                      [Shops, Outdoor, Services]
1165             [Shops, Eateries, Others, Services]
1166             [Services, Shops, Others, Eateries]
1167         [Eateries, Others, Services, Education]
1168                        [Others, Shops, Fitness]
1169                               [Eateries, Shops]
1170                       [Shops, Others, Eateries]
1171                       [Shops, Others, Eateries]
1172                                         [Shops]
1173                                     [Education]
1174                        [Shops, Others, Outdoor]
1175             [Others, Shops, Services, Eateries]
1176                            [Services, Eateries]
1177              [Outdoor, Others, Shops, Services]
1178                                     [Education]
1179                                 [Shops, Others]
1180                        [Outdoor, Shops, O

<ipython-input-48-af2ae68ea5b6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_record_df['categories'] = user_record_df['category_set'].apply(lambda x: list(pd.unique(x)))


In [18]:
def compute_general_venue_categories_count(result_df, df):
    print(result_df.user_id)
    user_record_df = df[df.user_id == result_df.user_id]
    user_record_df['categories'] = user_record_df['category_set'].apply(lambda x: list(pd.unique(x)))

    print(user_record_df.categories)

    categories = {}
    for record in user_record_df.categories:
        for x in record:
            if x not in categories.keys():
                categories[x] = 1
            else:
                categories[x] = categories[x] + 1

    print(categories)
    return categories
    

In [19]:
result_df['general_venue_categories'] = result_df.apply(compute_general_venue_categories_count, axis=1, args=[subsequence_all_df])

187
312                                 [Others]
313                                 [Others]
314                                 [Others]
315                                  [Shops]
316                       [Others, Eateries]
317                                 [Others]
318                       [Others, Eateries]
319                       [Others, Eateries]
320                          [Others, Shops]
321                       [Others, Eateries]
322                       [Others, Eateries]
323                       [Others, Eateries]
324                       [Others, Eateries]
325                       [Others, Eateries]
326                       [Others, Eateries]
327                           [Others, Arts]
328                  [Others, Arts, Outdoor]
329                        [Eateries, Shops]
330             [Others, Eateries, Services]
331                    [Eateries, Education]
332                       [Others, Eateries]
333                       [Others, Eateries]
334   

1286                             [Others, Shops, Outdoor]
1287                                            [Outdoor]
1288                                            [Outdoor]
1289                   [Others, Eateries, Shops, Outdoor]
1290                                            [Outdoor]
1291                                           [Services]
1292                          [Others, Eateries, Outdoor]
1293                                             [Others]
1294                   [Others, Eateries, Outdoor, Shops]
1295                           [Outdoor, Shops, Eateries]
1296                                   [Others, Eateries]
1297                                   [Others, Eateries]
1298                          [Others, Outdoor, Eateries]
1299                                    [Others, Outdoor]
1300                          [Eateries, Shops, Services]
1301                          [Others, Eateries, Outdoor]
1302                                   [Others, Eateries]
1303          

In [27]:
pd.set_option('display.max_colwidth', 1000)
display(result_df)

,user_id,dominant_sequence,time (s),general_venue_categories
0,187,[Others],1.314002,"{'Others': 44, 'Shops': 7, 'Eateries': 36, 'Arts': 2, 'Outdoor': 5, 'Services': 4, 'Education': 2}"
1,384,[],11.438999,"{'Others': 46, 'Eateries': 47, 'Shops': 30, 'Services': 32, 'Entertainment': 2, 'Education': 3, 'Arts': 5, 'Outdoor': 5, 'Fitness': 17}"
2,448,[],0.199000,"{'Eateries': 16, 'Shops': 35, 'Others': 36, 'Education': 13, 'Outdoor': 11, 'Services': 15, 'Fitness': 1}"
3,819,[Others],0.415029,"{'Eateries': 11, 'Outdoor': 35, 'Others': 46, 'Shops': 8, 'Services': 2, 'Fitness': 1}"
4,280,[Eateries],0.054999,"{'Eateries': 44, 'Shops': 21, 'Services': 8, 'Others': 11, 'Education': 10, 'Outdoor': 1, 'Entertainment': 1, 'Fitness': 1}"
5,281,[],0.106999,"{'Shops': 38, 'Eateries': 30, 'Outdoor': 5, 'Religious Places': 14, 'Others': 6, 'Education': 19, 'Fitness': 6, 'Entertainment': 3, 'Services': 1, 'Arts': 1}"
6,742,[Others],0.373035,"{'Others': 51, 'Shops': 26, 'Entertainment': 14, 'Eateries': 11, 'Outdoor': 18, 'Fitness': 13, 'Arts': 1, 'Services': 2}"
7,207,[],0.047000,"{'Others': 38, 'Shops': 40, 'Education': 4, 'Services': 2, 'Outdoor': 1}"
8,184,[],0.070990,"{'Eateries': 36, 'Others': 28, 'Arts': 9, 'Services': 8, 'Shops': 13, 'Entertainment': 3, 'Outdoor': 4, 'Education': 2, 'Fitness': 2}"
9,962,[Eateries],0.017995,"{'Outdoor': 2, 'Others': 39, 'Services': 2, 'Eateries': 14, 'Shops': 11, 'Fitness': 1}"


In [26]:
display(result_df[result_df.user_id == 448].general_venue_categories)

2    {'Eateries': 16, 'Shops': 35, 'Others': 36, 'Education': 13, 'Outdoor': 11, 'Services': 15, 'Fitness': 1}
Name: general_venue_categories, dtype: object